In [1]:
import pandas as pd
import random 
import json
import matplotlib.pyplot as plt

from functions import join_stocks_crypto, generate_rand_portfolios
from functions_post_clustering import dunn_bonferroni

c:\Users\Kirill\Documents\Projects\thesis\venv312\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [ ]:
#GET THE DATA IN

#Main part data 2022-2023
df_all_stocks = pd.read_csv('stocks_data_FINAL.csv',index_col='Date')
df_all_stocks.index = pd.to_datetime(df_all_stocks.index)
df_all_stocks.index = df_all_stocks.index.strftime('%Y-%m-%d')

cryptos_df = pd.read_csv('cryptos_data_new.csv', index_col='timestamp')
joined_df = join_stocks_crypto(cryptos_df, df_all_stocks, mode = 'stocks_left')
joined_df.index = pd.to_datetime(joined_df.index)
returns_all = joined_df.pct_change().dropna()

tickers_to_select = list(joined_df.columns)

#Out-of-sample data 2024:
df_stocks_24 = pd.read_csv('stocks_data_out_sample_2024_FINAL.csv',index_col='Date')
cryptos_df_24 = pd.read_csv('cryptos_data_out_sample_2024.csv', index_col='timestamp')
joined_df_24 = join_stocks_crypto(cryptos_df_24, df_stocks_24, mode = 'stocks_left')
joined_df_24 = joined_df_24[tickers_to_select]
joined_df_24.index = pd.to_datetime(joined_df_24.index)
returns_all_24 = joined_df_24.pct_change().dropna()


tickers = list(df_all_stocks.columns)

random.seed(42)
random_portfolios = generate_rand_portfolios(n_reps=1000, n_stocks=15, tickers=tickers)

with open('equalw_sets_new.json') as f:
    equalw_sets = json.load(f)

with open('mdr_reoptimized_sets_new_bounds.json') as f:
    mdr_reoptimized_sets = json.load(f)



C:\Users\Kirill\AppData\Local\Temp\ipykernel_14316\3916790673.py:20: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns_all_24 = joined_df_24.pct_change().dropna()


In [41]:
from functions_post_clustering import estimate_t_df_for_portfolio
import math

df_estimate = math.floor(estimate_t_df_for_portfolio(returns_all))
print(df_estimate)

Average df: 11.13, Min df: 5.71
11


SIMULATE AND EVALUATE

In [ ]:
#FULL CLAUDE GENERATION CODE
import numpy as np
import pandas as pd
from scipy.stats import mstats, normaltest
import random


def run_simulation(portfolio_dict, returns_for_portfolio, n_sims=100, t=100, 
                  distribution_model='multivar_norm', plot=False, initialPortfolio=100, 
                  winsorize=False, winsorize_limits=(0.01, 0.01), use_log_returns=False):
    returns_for_portfolio = returns_for_portfolio[list(portfolio_dict.keys())]
    
    if winsorize:
        winsorized_returns = returns_for_portfolio.copy()
        
        for col in winsorized_returns.columns:
            winsorized_returns[col] = mstats.winsorize(winsorized_returns[col], limits=winsorize_limits)
        
        returns_for_portfolio = winsorized_returns

    # Convert to log returns if specified (for consistency with out-of-sample)
    if use_log_returns:
        invalid_mask = returns_for_portfolio <= -1.0
        if invalid_mask.any().any():
            log_compatible_returns = returns_for_portfolio.copy()
            log_compatible_returns[invalid_mask] = -0.99
            log_returns = np.log(1 + log_compatible_returns)
            returns_for_portfolio = log_returns
        else:
            returns_for_portfolio = np.log(1 + returns_for_portfolio)

    mean_returns = returns_for_portfolio.mean()
    cov_matrix = returns_for_portfolio.cov()

    weights = np.array([v for _, v in portfolio_dict.items()])

    meanM = np.tile(mean_returns, (t, 1))  # Shape: (T, n_assets)

    portfolio_sims = np.zeros((t, n_sims))

    L = np.linalg.cholesky(cov_matrix)  # Cholesky decomposition

    for sim in range(n_sims):
        if distribution_model == 'bootstrap':
            sampled_returns = returns_for_portfolio.bfill().sample(n=t, replace=True).values
            portfolio_returns = sampled_returns @ weights
        elif distribution_model in ['multivar_norm', 'multivar_t']:
            if distribution_model == 'multivar_norm':
                Z = np.random.normal(size=(t, len(portfolio_dict)))  # Shape: (T, n_assets)
                daily_returns = meanM + Z @ L.T  # Shape: (T, n_assets)
            elif distribution_model == 'multivar_t':
                df = 11  # degrees of freedom
                Z = np.random.normal(size=(t, len(portfolio_dict)))
                chi2 = np.random.chisquare(df, size=(t, 1))
                Z_t = Z / np.sqrt(chi2 / df)  # now Z_t has t-distributed marginals

                daily_returns = meanM + Z_t @ L.T

            portfolio_returns = daily_returns @ weights  # Shape: (T,)
        else:
            break
            
        # Convert back from log returns if necessary
        if use_log_returns:
            portfolio_returns = np.exp(portfolio_returns) - 1
        
        portfolio_sims[:, sim] = np.cumprod(1 + portfolio_returns) * initialPortfolio

    return portfolio_sims

def out_of_sample_analysis(portfolio, return_data, start_date, period=126, 
                          calculate_metrics=True, plot_results=False, risk_free_rate=0.02, 
                          use_log_returns=False):
    
    if isinstance(start_date, str):
        start_date = pd.to_datetime(start_date)
    
    results = {}
    tickers = list(portfolio.keys())
    weights = np.array([portfolio[ticker] for ticker in tickers])
    
    # Validate that all tickers exist in the data
    missing_tickers = [ticker for ticker in tickers if ticker not in return_data.columns]
    if missing_tickers:
        raise ValueError(f"The following tickers are missing from return_data: {missing_tickers}")
    
    # Extract relevant data
    return_subset = return_data[tickers].copy()
    return_subset = return_subset.sort_index()
    oos_data = return_subset[return_subset.index >= start_date]
    
    if oos_data.empty:
        raise ValueError(f"No data available after start_date: {start_date}")
    
    # Get the specified period of data
    period_data = oos_data.iloc[:period]
    returns = period_data.copy()

    if use_log_returns:
        # Handle potential invalid values for log transformation
        invalid_mask = returns <= -1.0
        if invalid_mask.any().any():
            print(f"Warning: Found {invalid_mask.sum().sum()} invalid values for log transformation. Capping at -99%.")
            log_compatible_returns = returns.copy()
            log_compatible_returns[invalid_mask] = -0.99  # Cap extreme losses
            returns_for_portfolio = log_compatible_returns
        else:
            returns_for_portfolio = returns
        
        # CORRECTED: Calculate portfolio simple returns first
        portfolio_simple_returns = returns_for_portfolio.dot(weights)
        
        # Handle edge case where portfolio returns <= -1
        portfolio_invalid_mask = portfolio_simple_returns <= -1.0
        if portfolio_invalid_mask.any():
            print(f"Warning: Found {portfolio_invalid_mask.sum()} invalid portfolio returns for log transformation. Capping at -99%.")
            portfolio_simple_returns[portfolio_invalid_mask] = -0.99
        
        # Convert portfolio simple returns to log returns
        portfolio_log_returns = np.log(1 + portfolio_simple_returns)
        
        # Use simple returns for most calculations
        portfolio_returns = portfolio_simple_returns
        
        # Calculate cumulative returns using log returns (more numerically stable)
        cumulative_returns = np.exp(portfolio_log_returns.cumsum()) - 1
        
    else:
        # Standard simple returns approach
        portfolio_returns = returns.dot(weights)
        cumulative_returns = (1 + portfolio_returns).cumprod() - 1

    # Store basic results
    results = {
        'returns': portfolio_returns,
        'cumulative_returns': cumulative_returns,
        'period_data': period_data,
        'weights': weights,
        'tickers': tickers
    }
    
    if calculate_metrics:
        # Calculate performance metrics
        total_return = cumulative_returns.iloc[-1] if len(cumulative_returns) > 0 else 0
        trading_days = len(portfolio_returns)
        
        # Annualization factor based on actual trading days
        annual_factor = 252 / trading_days if trading_days > 0 else 1
        annualized_return = (1 + total_return) ** annual_factor - 1

        # Volatility calculations
        volatility = portfolio_returns.std() * np.sqrt(252) if len(portfolio_returns) > 1 else 0
        
        # Sharpe ratio calculations
        sharpe_ratio = (annualized_return - risk_free_rate) / volatility if volatility > 0 else 0
        
        # Non-annualized Sharpe ratio for the actual period
        rf_period = risk_free_rate * trading_days / 252
        volatility_period = portfolio_returns.std() * np.sqrt(trading_days) if len(portfolio_returns) > 1 else 0
        sharpe_period = (total_return - rf_period) / volatility_period if volatility_period > 0 else 0
        
        # Risk metrics
        var_95 = abs(portfolio_returns.quantile(0.05)) * 100 if len(portfolio_returns) > 0 else 0
        var_99 = abs(portfolio_returns.quantile(0.01)) * 100 if len(portfolio_returns) > 0 else 0
        
        # Maximum drawdown calculation
        rolling_max = cumulative_returns.expanding().max()
        drawdown = (cumulative_returns - rolling_max) / (1 + rolling_max)
        max_drawdown = abs(drawdown.min()) * 100 if len(drawdown) > 0 else 0
        
        # Additional metrics
        positive_days = (portfolio_returns > 0).sum() if len(portfolio_returns) > 0 else 0
        win_rate = positive_days / len(portfolio_returns) * 100 if len(portfolio_returns) > 0 else 0
        
        results.update({
            'mean_period_return': total_return,
            'annualised_return': annualized_return,
            'volatility': volatility,
            'sharpe_annualized': sharpe_ratio,
            'sharpe_period': sharpe_period,
            'var_95': var_95,
        })
    
  
    
    return results


def calculate_diversification_ratio(portfolio_dict, return_df):
    try:
        # Get the assets in the portfolio
        assets = list(portfolio_dict.keys())
        weights = np.array([portfolio_dict[asset] for asset in assets])
        
        # Filter return data to only include portfolio assets
        portfolio_returns = return_df[assets].dropna()
        
        if len(portfolio_returns) < 30:  # Need minimum data for reliable volatility estimates
            return np.nan
        
        # Calculate individual asset volatilities (annualized)
        individual_volatilities = portfolio_returns.std() * np.sqrt(252)
        
        # Calculate weighted average of individual volatilities
        weighted_avg_volatility = np.sum(weights * individual_volatilities)
        
        # Calculate portfolio returns
        portfolio_return_series = (portfolio_returns * weights).sum(axis=1)
        
        # Calculate portfolio volatility (annualized)
        portfolio_volatility = portfolio_return_series.std() * np.sqrt(252)
        
        # Calculate diversification ratio
        if portfolio_volatility > 0:
            diversification_ratio = weighted_avg_volatility / portfolio_volatility
            return diversification_ratio
        else:
            return np.nan
            
    except Exception as e:
        print(f"Error calculating diversification ratio: {e}")
        return np.nan


# Add this helper function
def calculate_cvar(returns, confidence_level=0.05):
    """
    Calculate Conditional Value at Risk (CVaR) - the expected loss beyond VaR
    """
    if len(returns) == 0:
        return np.nan
    
    var_threshold = np.percentile(returns, confidence_level * 100)
    tail_losses = returns[returns <= var_threshold]
    
    if len(tail_losses) == 0:
        return abs(var_threshold) * 100
    
    cvar = abs(np.mean(tail_losses)) * 100
    return cvar


def simulate_evaluate_portfolio_subset(portfolios_subset, return_df, n_sims=100, t=100, 
                                      distribution_model='multivar_norm', rf_annual=0.04, 
                                      seed=30, winsorize=False, winsorize_limits=(0.01, 0.01),
                                      use_log_returns=False):
    
    simulations_results_dict = dict()
    subset_statistics_df = pd.DataFrame()

    random.seed(seed)
    np.random.seed(seed)

    for i, portfolio_dict in portfolios_subset.items():
        # Making sure weights sum up to 1
        total = sum(portfolio_dict.values())
        portfolio_dict = {k: v / total for k, v in portfolio_dict.items()}
        
        # Calculate diversification ratio for this portfolio
        diversification_ratio = calculate_diversification_ratio(portfolio_dict, return_df)

        if distribution_model == 'out_sample_direct':
            # Out-of-sample analysis
            res = out_of_sample_analysis(
                portfolio_dict, 
                return_df, 
                '2024-01-01', 
                period=t, 
                calculate_metrics=True, 
                risk_free_rate=rf_annual, 
                plot_results=False, 
                use_log_returns=use_log_returns
            )
            
            mean_annual_return = res['annualised_return']
            mean_return = res['mean_period_return']
            sharpe_annual = res['sharpe_annualized']
            sharpe_period = res['sharpe_period']
            
            # Bootstrap period VaR/CVaR for consistency with Monte Carlo
            portfolio_returns = res['returns']  # Daily returns over the period
            n_bootstrap = 500
            bootstrap_period_returns = []
            
            np.random.seed(seed)  # For reproducibility
            for _ in range(n_bootstrap):
                # Sample with replacement from actual daily returns
                sampled_returns = np.random.choice(portfolio_returns, size=len(portfolio_returns), replace=True)
                # Calculate cumulative return for this bootstrap sample
                bootstrap_period_return = (1 + sampled_returns).prod() - 1
                bootstrap_period_returns.append(bootstrap_period_return)
            
            bootstrap_period_returns = np.array(bootstrap_period_returns)
            
            # Calculate period VaR and CVaR from bootstrap
            period_var_95 = abs(np.percentile(bootstrap_period_returns, 5)) * 100
            period_cvar_95 = calculate_cvar(bootstrap_period_returns, confidence_level=0.05)
            
            # Annualize VaR/CVaR for cross-period comparison
            actual_period_days = len(portfolio_returns)
            annualized_var = period_var_95 * np.sqrt(252 / actual_period_days)
            annualized_cvar = period_cvar_95 * np.sqrt(252 / actual_period_days)
            
            # Calculate annualized volatility for consistency
            daily_volatility = portfolio_returns.std()
            annualized_volatility = daily_volatility * np.sqrt(252)

            stat_results = pd.DataFrame({
                'annualised_return': [mean_annual_return],
                'mean_period_return': [mean_return],
                'sharpe_annualized': [sharpe_annual],
                'sharpe_period': [sharpe_period],
                'period_VaR': [period_var_95],
                'period_CVaR': [period_cvar_95],
                'annualized_VaR': [annualized_var],
                'annualized_CVaR': [annualized_cvar],
                'annualized_volatility': [annualized_volatility],
            })
            
            subset_statistics_df = pd.concat([subset_statistics_df, stat_results])
            
        else:
            # Monte Carlo simulation
            portfolio_sims = run_simulation(
                portfolio_dict, 
                return_df[list(portfolio_dict.keys())], 
                n_sims=n_sims, 
                t=t, 
                distribution_model=distribution_model, 
                plot=False, 
                winsorize=winsorize, 
                winsorize_limits=winsorize_limits,
                use_log_returns=use_log_returns
            )

            simulations_results_dict[i] = portfolio_sims
            actual_period_days = t
            
            # Calculate total period returns across all simulations
            initial_values = portfolio_sims[0, :]  # Should be 100 for all simulations
            final_values = portfolio_sims[-1, :]   # Final portfolio values
            total_returns_array = (final_values - initial_values) / initial_values
            
            # Calculate daily returns for volatility calculations
            daily_returns = (portfolio_sims[1:, :] - portfolio_sims[:-1, :]) / portfolio_sims[:-1, :]
            
            # Calculate period VaR and CVaR from total returns across simulations
            period_var_95 = abs(np.percentile(total_returns_array, 5)) * 100
            period_cvar_95 = calculate_cvar(total_returns_array, confidence_level=0.05)
            
            # Annualize VaR/CVaR for cross-period comparison
            annualized_var = period_var_95 * np.sqrt(252 / actual_period_days)
            annualized_cvar = period_cvar_95 * np.sqrt(252 / actual_period_days)
            
            # Calculate metrics for each simulation path
            all_stats = []
            
            for sim in range(n_sims):
                # Get daily returns for this simulation
                sim_daily_returns = daily_returns[:, sim]
                total_return = total_returns_array[sim]
                
                # Annualization factor based on full simulation period
                annual_factor = 252 / actual_period_days
                annualized_return = (1 + total_return) ** annual_factor - 1
                
                # Calculate volatility (annualized) from daily returns
                daily_volatility = sim_daily_returns.std()
                annualized_volatility = daily_volatility * np.sqrt(252)
                
                # Calculate Sharpe ratios
                sharpe_annual = (annualized_return - rf_annual) / annualized_volatility if annualized_volatility > 0 else 0
                
                # Period Sharpe ratio - use consistent period length
                rf_period = rf_annual * actual_period_days / 252
                period_volatility = daily_volatility * np.sqrt(actual_period_days)
                sharpe_period = (total_return - rf_period) / period_volatility if period_volatility > 0 else 0
                
                all_stats.append({
                    'annualised_return': annualized_return,
                    'mean_period_return': total_return,
                    'sharpe_annualized': sharpe_annual,
                    'sharpe_period': sharpe_period,
                    'annualized_volatility': annualized_volatility
                })
            
            # Convert to DataFrame and calculate averages
            sim_stats_df = pd.DataFrame(all_stats)
            
            mean_annual_return = sim_stats_df['annualised_return'].mean()
            mean_return = sim_stats_df['mean_period_return'].mean()
            mean_sharpe_annual = sim_stats_df['sharpe_annualized'].mean()
            mean_sharpe_period = sim_stats_df['sharpe_period'].mean()
            mean_volatility = sim_stats_df['annualized_volatility'].mean()
            
            stat_results = pd.DataFrame({
                'annualised_return': [mean_annual_return],
                'mean_period_return': [mean_return],
                'sharpe_annualized': [mean_sharpe_annual],
                'sharpe_period': [mean_sharpe_period],
                'period_VaR': [period_var_95],
                'period_CVaR': [period_cvar_95],
                'annualized_VaR': [annualized_var],
                'annualized_CVaR': [annualized_cvar],
                'annualized_volatility': [mean_volatility],
            })

            subset_statistics_df = pd.concat([subset_statistics_df, stat_results])

    subset_statistics_df = subset_statistics_df.reset_index(drop=True)

    # Run normality test on all metrics
    results_normality_test = {}
    for col in subset_statistics_df.columns:
        if subset_statistics_df[col].dtype in ['float64', 'int64']:  # Only test numeric columns
            stat, p_value = normaltest(subset_statistics_df[col])
            results_normality_test[col] = {'statistic': stat, 'p_value': p_value}

    normality_results_df = pd.DataFrame(results_normality_test).T
    normality_results_df['normal'] = normality_results_df['p_value'] > 0.05

    return simulations_results_dict, subset_statistics_df, normality_results_df

In [6]:
import os
folder_overall = 'NEW_RESULTS_FINAL/'
n_sims = 1000
data_returns = returns_all
data_name = 'simulation'
distribution_models = ['bootstrap', 'multivar_t']
time_periods = [126, 189, 252]
portfolio_sets_group = {'mdr_sets': mdr_reoptimized_sets, 'equalw_sets': equalw_sets}
windsorize = False
folder = 'simulation_results_FINAL_new_bounds'


MODE = 'OUT_OF_SAMPLE_DIRECT' 


if MODE == 'OUT_OF_SAMPLE_DIRECT':
    data_returns = returns_all_24
    data_name = 'out_sample_direct'
    distribution_models = ['out_sample_direct']
    windsorize = False
    folder = 'out_of_sample_direct_results_FINAL_new_bounds'

folder = folder_overall + folder
# Make sure output folders exist
os.makedirs(f'{folder}', exist_ok=True)

for set_name, set_itself in portfolio_sets_group.items():
    for distribution_model in distribution_models:
        for time_period in time_periods:
            subset_statistics_results_dfs = dict()
            normality_results_dfs = dict()
            results_all_df = pd.DataFrame()

            for key, portfolio_set in set_itself.items():
                simulations_results_dict, subset_statistics_df, normality = simulate_evaluate_portfolio_subset(
                    portfolio_set,
                    data_returns,
                    n_sims=n_sims,
                    t=time_period,
                    distribution_model=distribution_model,
                    winsorize=windsorize,
                    winsorize_limits=(0.01, 0.01),
                    use_log_returns=True
                )

                # Initialize nested dicts
                subset_statistics_results_dfs.setdefault(time_period, {})
                normality_results_dfs.setdefault(time_period, {})

                subset_statistics_results_dfs[time_period][key] = subset_statistics_df
                normality_results_dfs[time_period][key] = normality

                # Mean series and concat
                mean_series = subset_statistics_df.mean()
                mean_df = pd.DataFrame(mean_series, columns=[(time_period, key)])
                mean_df.columns = pd.MultiIndex.from_tuples(mean_df.columns)
                results_all_df = pd.concat([results_all_df, mean_df], axis=1)

            # Save to CSV with clear naming
            csv_filename = f"{folder}/stats_{set_name}_{distribution_model}_t{time_period}.csv"
            results_all_df.to_csv(csv_filename)

            # Run and save Dunn-Bonferroni tests
            all_dunn_results = dict()
            dunn_bonferroni_test_results = dunn_bonferroni(subset_statistics_results_dfs[time_period], metrics='all')
            all_dunn_results[time_period] = dunn_bonferroni_test_results

            excel_filename = f"{folder}/dunn_matrix_{set_name}_{distribution_model}_t{time_period}.xlsx"
            with pd.ExcelWriter(excel_filename) as writer:
                for sheet_name, df in all_dunn_results[time_period].items():
                    df.to_excel(writer, sheet_name=sheet_name[:31])


            print(f'Done for {set_name}_{distribution_model}_t{time_period}, moving on')

Done for mdr_sets_out_sample_direct_t126, moving on
Done for mdr_sets_out_sample_direct_t189, moving on
Done for mdr_sets_out_sample_direct_t252, moving on
Done for equalw_sets_out_sample_direct_t126, moving on
Done for equalw_sets_out_sample_direct_t189, moving on
Done for equalw_sets_out_sample_direct_t252, moving on
